In [4]:
import numpy as np
import pandas as pd

In [5]:
import os
os.chdir("../../evaluation_weighted/results")
os.getcwd()

'D:\\projekty\\2020\\LadaM_GroupRecs\\evaluation_weighted\\results'

In [6]:
def compute_metrics(alg_name, groups, results):
    # test_data are triplets: user_id, item_id, and rating
    #LP: test data is matrix user_id x item_id !!!!!! a ja si rikal, jakto ze ti to prirazeni funguje...
    estimated_rating = []
    true_rating = []
    diff_rating = []

    
    estimated_nDCG = []
    true_nDCG = []
    diff_nDCG = []

        
    res_ndcg = results.loc[(results.metric == "nDCG")]
    res_AR = results.loc[(results.metric == "AR")] 
        
        #print(gid_res_ndcg)
        #print(gid_res_AR)
        
    users_AR_ratings = res_AR.result.values
    users_ndcg_ratings = res_ndcg.result.values

    users_AR_true_share = res_AR.expected_share.values
    users_ndcg_true_share = res_ndcg.expected_share.values    
    
    users_AR_result_share = res_AR.result_share.values
    users_ndcg_result_share = res_ndcg.result_share.values     
    
    users_AR_diff = np.absolute(users_AR_true_share - users_AR_result_share)
    users_ndcg_diff = np.absolute(users_ndcg_true_share - users_ndcg_result_share)   
      
    results = pd.DataFrame({
        "AR_avg": users_AR_ratings ,
        "AR_diff": users_AR_diff ,
        
        "nDCG_avg": users_ndcg_ratings ,
        "ndcg_diff": users_ndcg_diff 

    })
      
    return results

In [7]:
from scipy import stats
#Comparison for size 8
def calculate_significance(results, grouptype, size):
    print("")
    print(grouptype,size)

    otherAlgs = [a for a in algs if ( grouptype+"_group_"+size in a )]

    for count1, alg1 in enumerate(otherAlgs):
        baseDF = results[alg1]
        
        for count2,alg in enumerate(otherAlgs):
            if count1 != count2:
                otherDF = results[alg]
                res = {}
                for col in otherDF.columns:
                    mean_diff = np.mean(baseDF.loc[:,col] - otherDF.loc[:,col])
                    stat,pval = stats.ttest_rel(baseDF.loc[:,col], otherDF.loc[:,col])
                    if pval > 0.0001:
                        print("pval_excess",col)
                    res[col] = (mean_diff, pval)

                alg1 = alg1.replace("mf_30_1.0_avg_","")
                alg1 = alg1.replace(grouptype+"_group_"+size+"_","")
                alg = alg.replace("mf_30_1.0_avg_","")
                alg = alg.replace(grouptype+"_group_"+size+"_","")
                print(alg1+" - "+ alg)
                print(pd.DataFrame(res, index=["diff","pval"]))
                print("")

In [8]:
results_all = {}
for grouptype in ["sim","div","random"]:
    for size in ["2","3","4","8"]:
        fileName = "result_"+grouptype+"_"+size
        resultsDF = pd.read_csv(fileName, sep=",", header=0)
        algs = resultsDF.alg.unique()
        metrics = resultsDF.metric.unique()
        groups = resultsDF.group_id.unique()
        results = {}
        for a in algs:
            resPerAlg = resultsDF.loc[resultsDF.alg == a]
            results[a] = compute_metrics(a, metrics, resPerAlg)
        calculate_significance(results, grouptype, size)



sim 2
GFAR_1.0 - pref_AVG
             AR_avg   AR_diff       nDCG_avg  ndcg_diff
diff  -2.668124e-01  0.063812  -5.866018e-02   0.075694
pval  4.775488e-293  0.000000  5.710212e-218   0.000000

GFAR_1.0 - pref_FuzzyDHondtDirectOptimize_1.0
            AR_avg   AR_diff      nDCG_avg  ndcg_diff
diff -1.079100e-01  0.130472 -2.382668e-02   0.109605
pval  1.379213e-35  0.000000  4.808216e-27   0.000000

GFAR_1.0 - pref_FuzzyDHondt_1.0
        AR_avg  AR_diff  nDCG_avg  ndcg_diff
diff -0.295388  0.05826 -0.068602   0.063323
pval  0.000000  0.00000  0.000000   0.000000

pref_AVG - GFAR_1.0
             AR_avg   AR_diff       nDCG_avg  ndcg_diff
diff   2.668124e-01 -0.063812   5.866018e-02  -0.075694
pval  4.775488e-293  0.000000  5.710212e-218   0.000000

pref_AVG - pref_FuzzyDHondtDirectOptimize_1.0
             AR_avg  AR_diff       nDCG_avg  ndcg_diff
diff   1.589024e-01  0.06666   3.483350e-02   0.033911
pval  1.433153e-234  0.00000  3.576897e-186   0.000000

pref_AVG - pref_FuzzyDHond


div 2
GFAR_1.0 - pref_AVG
            AR_avg   AR_diff      nDCG_avg  ndcg_diff
diff -1.246841e-01  0.070506 -2.219793e-02   0.107236
pval  2.211522e-45  0.000000  6.671702e-16   0.000000

pval_excess AR_avg
GFAR_1.0 - pref_FuzzyDHondtDirectOptimize_1.0
        AR_avg   AR_diff      nDCG_avg  ndcg_diff
diff  0.028050  0.181139 -1.618955e-02   0.124228
pval  0.001144  0.000000  1.504811e-09   0.000000

GFAR_1.0 - pref_FuzzyDHondt_1.0
             AR_avg   AR_diff       nDCG_avg  ndcg_diff
diff  -1.448319e-01  0.098022  -4.731511e-02     0.1154
pval  2.108809e-113  0.000000  1.353746e-117     0.0000

pref_AVG - GFAR_1.0
            AR_avg   AR_diff      nDCG_avg  ndcg_diff
diff  1.246841e-01 -0.070506  2.219793e-02  -0.107236
pval  2.211522e-45  0.000000  6.671702e-16   0.000000

pval_excess nDCG_avg
pref_AVG - pref_FuzzyDHondtDirectOptimize_1.0
             AR_avg   AR_diff  nDCG_avg      ndcg_diff
diff   1.527346e-01  0.110632  0.006008   1.699198e-02
pval  5.623598e-105  0.000000  0.


random 2
GFAR_1.0 - pref_AVG
             AR_avg        AR_diff      nDCG_avg  ndcg_diff
diff  -1.740447e-01   5.635973e-02 -3.580212e-02    0.08684
pval  5.538663e-100  2.192184e-303  8.435950e-48    0.00000

pval_excess AR_avg
GFAR_1.0 - pref_FuzzyDHondtDirectOptimize_1.0
        AR_avg   AR_diff      nDCG_avg  ndcg_diff
diff -0.017788  0.160715 -2.248648e-02    0.10853
pval  0.033298  0.000000  1.320833e-19    0.00000

GFAR_1.0 - pref_FuzzyDHondt_1.0
             AR_avg   AR_diff       nDCG_avg  ndcg_diff
diff  -1.943862e-01  0.078103  -5.642160e-02   0.092794
pval  9.882142e-232  0.000000  2.478885e-208   0.000000

pref_AVG - GFAR_1.0
             AR_avg        AR_diff      nDCG_avg  ndcg_diff
diff   1.740447e-01  -5.635973e-02  3.580212e-02   -0.08684
pval  5.538663e-100  2.192184e-303  8.435950e-48    0.00000

pref_AVG - pref_FuzzyDHondtDirectOptimize_1.0
             AR_avg   AR_diff      nDCG_avg      ndcg_diff
diff   1.562569e-01  0.104355  1.331565e-02   2.168950e-02
pval  7